## **Hemorrhage Classifier:**

In [6]:
import torch.nn as nn
import torchvision.models

alexnet_model = torchvision.models.alexnet(pretrained=True)
alexnet_model.features[0] = nn.Conv2d(1, 64, kernel_size= 7, stride= 2, padding= 3)

class HemorrhageClassifier(nn.Module):
    def __init__(self):
        super(HemorrhageClassifier, self).__init__()
        self.name = "Classifier"

        for param in alexnet_model.parameters():
          param.requires_grad = False

        self.fc1 = nn.Linear(512 * 512 * 1, 100)
        self.fc2 = nn.Linear(100, 5)

    def forward(self, x):
        x = alexnet_model.features(x)
        x = x.view(-1, 512 * 512 * 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /home/addy/.cache/torch/checkpoints/alexnet-owt-4df8aa71.pth
51.9%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

60.7%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

70.2%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iop

In [15]:
import torch.nn as nn
import torchvision.models

resnet152 = torchvision.models.resnet152(pretrained=True)
modules = list(resnet152.children())
modules[0].in_channels = 1

class HemorrhageClassifier2(nn.Module):
    def __init__(self):
        super(HemorrhageClassifier2, self).__init__()
        self.name = "Classifier 2"

        for param in resnet152.parameters():
          param.requires_grad = False

        self.fc1 = nn.Linear(512 * 512 * 1, 100)
        self.fc2 = nn.Linear(100, 5)

    def forward(self, x):
        x = resnet152(x)
        x = x.view(-1, 512 * 512 * 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## **Hemorrhage Detector**

**Two convulational layers:**
  - each have a 5x5 receptive field, 2x2 stride, and 2x2 padding

**The number of feature maps:**
  - for the 1st convulational layer is 10
  - for the 2nd convolutional layer is 20

**Two Pooling Layers:**
  - Receptive field: 2x2, stride: 2x2

**Fully Connected layer:**
  - 100 nodes
  - 2 output nodes (binary classification)

**Training Parameters:**

lr = 0.02
momentum =0.01
4000 iterations (60 epochs)
Batch size of 20
Stochastic gradient descent

In [21]:
import torch.nn as nn
import torchvision.models

alexnet_model = torchvision.models.alexnet(pretrained=True)
alexnet_model.features[0] = nn.Conv2d(1, 64, kernel_size= 7, stride= 2, padding= 3)

class HemorrhageDetector(nn.Module):
    def __init__(self):
        super(HemorrhageDetector, self).__init__()
        self.name = "Detector"

        for param in alexnet_model.parameters():
          param.requires_grad = False

        self.fc1 = nn.Linear(512 * 512 * 1, 100)
        self.fc2 = nn.Linear(100, 2)

    def forward(self, x):
        x = alexnet_model.features(x)
        x = x.view(-1, 512 * 512 * 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

In [22]:
import torch.nn as nn
import torchvision.models

resnet152 = torchvision.models.resnet152(pretrained=True)
modules = list(resnet152.children())
modules[0].in_channels = 1

class HemorrhageDetector2(nn.Module):
    def __init__(self):
        super(HemorrhageDetector2, self).__init__()
        self.name = "Detector 2"

        for param in resnet152.parameters():
          param.requires_grad = False

        self.fc1 = nn.Linear(512 * 512 * 1, 100)
        self.fc2 = nn.Linear(100, 2)

    def forward(self, x):
        x = resnet152(x)
        x = x.view(-1, 512 * 512 * 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

**Training**

In [19]:
def train(model, dataset, batch_size = 64, learning_rate=0.01, num_epochs=30):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    
    iters, losses, train_acc, val_acc = [], [], [], []
    
    training_loader = torch.utils.data.DataLoader(dataset, batch_size= batch_size)

    n = 0 
    for epoch in range(num_epochs):
        count = 0
        for imgs, labels in iter(training_loader):
            imgs = features = torch.load(f'training_features_batchcount{count}.tensor')
            imgs = torch.from_numpy(imgs.detach().numpy())

            #To Enable GPU Usage
            if use_cuda and torch.cuda.is_available():
              imgs = imgs.cuda()
              labels = labels.cuda()
            #############################################
            
            outputs = model(imgs)             
            loss = criterion(outputs, labels) 
            loss.backward()               
            optimizer.step()              
            optimizer.zero_grad()         

            # save the current training information
            iters.append(n)
            losses.append(float(loss)/batch_size)             # finding the average loss
            train_acc.append(get_alex_accuracy(model, train=True)) # finding the training accuracy 
            val_acc.append(get_alex_accuracy(model, train=False))  # finding the validation accuracy
            n = n+1
            count = count+1
        print("Epoch: ",epoch,". Training Accuracy: ", train_acc[n-1],". Validation Accuracy: ",val_acc[n-1])

    plt.title("Training Loss Curve")
    plt.plot(iters, losses, label="Train")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training vs. Validation Accuracy Curves")
    plt.plot(iters, train_acc, label="Train")
    plt.plot(iters, val_acc, label="Validation")
    plt.xlabel("Iterations")
    plt.ylabel("Accuracy")
    plt.legend(loc='best')
    plt.show()